#### 1. Imports

In [ ]:
import numpy as np
import pandas as pd
from math import e
from scipy.stats import chi2
from scipy.stats import laplace
from sklearn.metrics import mean_squared_error, mean_absolute_error
from matplotlib import pyplot as plt
from funcoes import gradient_descent, l2_gradient, proj, min_l2_norm, gerarTabelas, mre, chi, pValue, KL, fisher, cochran

---
#### 2. Dataset and original values

In [ ]:
dataset = pd.read_csv('datasetReal.csv').values

In [ ]:
N = dataset.shape[0]

In [ ]:
tabsCont = gerarTabelas(dataset)

In [ ]:
quiQuadrado= chi(tabsCont)

In [ ]:
pValor = pValue(quiQuadrado, 2)

In [ ]:
logPValor = np.log10(pValor)

In [ ]:
budgets = [0.1, 0.5, 1, 2, 5, 7, 10]

---
#### 3. CTPriv approach (noise in contingency table) - Geometric Mechanism

In [ ]:
errosMSE = np.zeros((10, len(budgets)))
errosMAE = np.zeros((10, len(budgets)))
errosMRE = np.zeros((10, len(budgets)))
divergKL = np.zeros((10, len(budgets)))

In [ ]:
sensibilidadeGeom = 1
for ep in range(0, len(budgets)):
    for i in range(0, 10):
        tabelasRuidosas = []
        for j in range(0, len(tabsCont)):
            tabela = np.array(tabsCont[j])
            p = 1 - np.exp([[-budgets[ep],-budgets[ep],-budgets[ep]],[-budgets[ep],-budgets[ep],-budgets[ep]]])
            ruidoGeo = np.random.geometric(p, size=tabela.shape) - np.random.geometric(p, size=tabela.shape)
            tabelaGeo = tabela + ruidoGeo
            desired_sum = np.sum(tabelaGeo, axis=1)
            for linha in range(0, tabelaGeo.shape[0]):
                tabelaGeo[linha] = min_l2_norm(tabelaGeo[linha], desired_sum[linha], min_value=0)
            tabelasRuidosas.append(tabelaGeo)
            if((j+1)%1000==0):
                print('SNP: ', (j+1))
        tabelasRuidosas = np.array(tabelasRuidosas)
        qui = chi(tabelasRuidosas)
        p = pValue(qui, 2)
        logs = []
        for pv in p:
            if(pv==0):
                logs.append(0)
            else:
                logs.append(np.log10(pv))
        log = np.array(logs)
        errosMSE[i, ep] = mean_squared_error(log, logPValor)
        errosMAE[i, ep] = mean_absolute_error(log, logPValor)
        errosMRE[i, ep] = mre(log, logPValor)
        divergKL[i, ep] = KL(logPValor, log)
    print('Epsilon: ',budgets[ep], ' -> OK')

---
#### 4. Approach Yamamoto et. al. (noise in metrics) - Laplace Mechanism

In [ ]:
errosMSEY = np.zeros((10, len(budgets)))
errosMAEY = np.zeros((10, len(budgets)))
errosMREY = np.zeros((10, len(budgets)))
divergKLY = np.zeros((10, len(budgets)))

In [ ]:
sensibilidadeY = np.log10(e)*(2*N)/(N+2)
for ep in range(0, len(budgets)):
    for i in range(0, 10):
        ruido = laplace.rvs(loc = 0, scale = sensibilidadeY/budgets[ep], size = logPValor.shape[0])
        logPValorRuid = logPValor + ruido
        min = 1
        for j in range(0, len(tabsCont)):
            if(logPValorRuid[i] > 0 and logPValorRuid[i] < min):
                min = logPValorRuid[i]
        for j in range(0, len(tabsCont)):
            if(logPValorRuid[j] < 0):
                logPValorRuid[j] = min
        errosMSEY[i, ep] = mean_squared_error(logPValorRuid, logPValor)
        errosMAEY[i, ep] = mean_absolute_error(logPValorRuid, logPValor)
        errosMREY[i, ep] = mre(logPValorRuid, logPValor)
        divergKLY[i, ep] = KL(-logPValor, logPValorRuid)
    print('Epsilon: ',budgets[ep], ' -> OK')

---
#### 5. Comparison of errors

In [ ]:
errosMSEAverage = np.average(errosMSE, axis = 0)
errosMAEAverage = np.average(errosMAE, axis = 0)
errosMREAverage = np.average(errosMRE, axis = 0)
divergKLAverage = np.average(divergKL, axis = 0)

In [ ]:
errosMSEYAverage = np.average(errosMSEY, axis = 0)
errosMAEYAverage = np.average(errosMAEY, axis = 0)
errosMREYAverage = np.average(errosMREY, axis = 0)
divergKLYAverage = np.average(divergKLY, axis = 0)

In [ ]:
plt.figure(figsize=(15,8))
bud = ['0.1', '0.5', '1', '2', '5', '7', '10']
plt.plot(bud, errosMSEAverage)
plt.plot(bud, errosMSEAverage, 'x', color='blue', label='CTPriv')
plt.plot(bud, errosMSEYAverage)
plt.plot(bud, errosMSEYAverage, 'x', color='orange', label='Yamamoto et. al.')
plt.legend()
plt.xlabel('Budgets')
plt.ylabel('MSE')
plt.savefig('Dataset Real/MSE - Comparison - Chi.png')

In [ ]:
plt.figure(figsize=(15,8))
plt.plot(bud, errosMAEAverage)
plt.plot(bud, errosMAEAverage, 'x', color='blue', label='CTPriv')
plt.plot(bud, errosMAEYAverage)
plt.plot(bud, errosMAEYAverage, 'x', color='orange', label='Yamamoto et. al.')
plt.legend()
plt.xlabel('Budgets')
plt.ylabel('MAE')
plt.savefig('Dataset Real/MAE - Comparison - Chi.png')

In [ ]:
plt.figure(figsize=(15, 8))
plt.plot(bud, errosMREAverage)
plt.plot(bud, errosMREAverage, 'x', color='blue', label='CTPriv')
plt.plot(bud, errosMREYAverage)
plt.plot(bud, errosMREYAverage, 'x', color='orange', label='Yamamoto et. al.')
plt.legend()
plt.xlabel('Budgets')
plt.ylabel('MRE')
plt.savefig('Dataset Real/MRE - Comparison - Chi.png')

In [ ]:
plt.figure(figsize=(15,8))
plt.plot(bud, np.log10(errosMSEAverage))
plt.plot(bud, np.log10(errosMSEAverage), 'x', color='blue', label='CTPriv')
plt.plot(bud, np.log10(errosMSEYAverage))
plt.plot(bud, np.log10(errosMSEYAverage), 'x', color='orange', label='Yamamoto et. al.')
plt.legend()
plt.xlabel('Budgets')
plt.ylabel('MSE')
plt.savefig('Dataset Real/MSE - Comparison - Chi - Log.png')

In [ ]:
plt.figure(figsize=(15,8))
plt.plot(bud, np.log10(errosMAEAverage))
plt.plot(bud, np.log10(errosMAEAverage), 'x', color='blue', label='CTPriv')
plt.plot(bud, np.log10(errosMAEYAverage))
plt.plot(bud, np.log10(errosMAEYAverage), 'x', color='orange', label='Yamamoto et. al.')
plt.legend()
plt.xlabel('Budgets')
plt.ylabel('MAE')
plt.savefig('Dataset Real/MAE - Comparison - Chi - Log.png')

In [ ]:
plt.figure(figsize=(15, 8))
plt.plot(bud, np.log10(errosMREAverage))
plt.plot(bud, np.log10(errosMREAverage), 'x', color='blue', label='CTPriv')
plt.plot(bud, np.log10(errosMREYAverage))
plt.plot(bud, np.log10(errosMREYAverage), 'x', color='orange', label='Yamamoto et. al.')
plt.legend()
plt.xlabel('Budgets')
plt.ylabel('MRE')
plt.savefig('Dataset Real/MRE - Comparison - Chi - Log.png')

In [ ]:
plt.figure(figsize=(15, 8))
plt.plot(bud, divergKLAverage)
plt.plot(bud, divergKLAverage, 'x', color='blue', label='CTPriv')
plt.plot(bud, divergKLYAverage)
plt.plot(bud, divergKLYAverage, 'x', color='orange', label='Yamamoto et. al.')
plt.legend()
plt.xlabel('Budgets')
plt.ylabel('KL Divergence')
plt.savefig('Dataset Real/KL - Comparison - Chi.png')

In [ ]:
with open("Dataset Real/Erros - Chi.txt", "w") as text_file:
    for i in range(0, len(budgets)):
        text_file.write('Epsilon: '+str(budgets[i])+'\n')
        text_file.write('   +MSE:\n')
        text_file.write('      ->CTPriv: '+str(errosMSEAverage[i])+'\n')
        text_file.write('      ->Yamamoto: '+str(errosMSEYAverage[i])+'\n')
        text_file.write('   +MAE:\n')
        text_file.write('      ->CTPriv: '+str(errosMAEAverage[i])+'\n')
        text_file.write('      ->Yamamoto: '+str(errosMAEYAverage[i])+'\n')
        text_file.write('   +MRE:\n')
        text_file.write('      ->CTPriv: '+str(errosMREAverage[i])+'\n')
        text_file.write('      ->Yamamoto: '+str(errosMREYAverage[i])+'\n')
        text_file.write('   +KL Divergência:\n')
        text_file.write('      ->CTPriv: '+str(divergKLAverage[i])+'\n')
        text_file.write('      ->Yamamoto: '+str(divergKLYAverage[i])+'\n')
text_file.close()